<a href="https://colab.research.google.com/github/Sriramsai-Mutyala/multilingual-ner/blob/main/sriramsai_pynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
np.random.seed(0)

In [ ]:
import pandas as pd
import zipfile

# Specify the path to the ZIP file
zip_file_path = '/content/archive (1) (1).zip'
# Specify the name of the CSV file you want to read
csv_file_name = 'tamil_ner.csv'

# Open the ZIP file and read the specified CSV file
with zipfile.ZipFile(zip_file_path, 'r') as zip_file:
    with zip_file.open(csv_file_name) as csv_file:
        data1 = pd.read_csv(csv_file, encoding='UTF-8')

# Display the first 10 rows of the DataFrame
data1.head(10)

,name,tags
0,கருநாடக,NN
1,அரசின்,NN
2,வேண்டுகோளின்படி,NN
3,இந்த,DMD
4,படைத்துறை,NN
5,விமானத்தளம்,NN
6,இந்திய,JJ
7,அரசால்,NN
8,குடிசார்,NN
9,பயன்பாட்டிற்கு,VNG


In [ ]:
data1.isnull().sum()


name        0
tags    44677
dtype: int64

In [ ]:
print("Unique Words in corpus:", data1['product_name'].nunique())
print("Unique Tag in corpus:", data1['tags'].nunique())


NameError: ignored

In [ ]:
print("Unique Words in corpus:",data1['name'].nunique())
print("Unique Tag in corpus:",data1['tags'].nunique())

Unique Words in corpus: 89638
Unique Tag in corpus: 29


In [ ]:
print(data1.columns)

Index(['name', 'tags'], dtype='object')


In [ ]:
words = list(set(data1['name'].values))
words.append("ENDPAD")
num_words = len(words)
tags = list(set(data1['tags'].values))
num_tags = len(tags)
num_words,num_tags

(89639, 30)

In [ ]:
import pandas as pd

# Assuming 'Tag' is the column containing tags in your DataFrame
tags = list(data1['tags'].values)
tags.append("ENDPAD")

num_tags = data1['tags'].nunique()  # Number of unique tags excluding 'ENDPAD'
tag_values = data1['tags'].unique()  # Unique tag values excluding 'ENDPAD'

# Filter out 'ENDPAD' if it exists
tag_values = [tag for tag in tag_values if tag != 'ENDPAD']

print("Number of Tags:", num_tags)
print("Tag Values (excluding 'ENDPAD'):", tag_values)

Number of Tags: 29
Tag Values (excluding 'ENDPAD'): ['NN', 'DMD', 'JJ', 'VNG', nan, 'PUNC', 'CCS_UT', 'PSP', 'VAUX', 'PRP', 'QTF', 'VINF', 'VNF', 'NNP', 'CSD', 'RB', 'QTO', 'QTC', 'VF', 'CCS', 'NST', 'SYM', 'INTF', 'NNV', 'PRF', 'DMQ', 'VN', 'PRQ', 'NEG', 'RPD']


In [ ]:
import pandas as pd

# Assuming 'Tag' is the column containing tags in your DataFrame
tags = list(data1['tags'].values)
tags.append("ENDPAD")

# Count the occurrences of each tag
tag_counts = pd.Series(tags).value_counts()

# Remove the count for 'ENDPAD' if it exists
tag_counts = tag_counts.drop('ENDPAD', errors='ignore')

# Print the total number of tags
total_tags = tag_counts.sum()
print("Total Number of Tags:", total_tags)

# Print the tag counts
print("Tag Counts:")
print(tag_counts)

Total Number of Tags: 497548
Tag Counts:
NN        285875
PUNC       32461
QTC        30328
NNP        29641
VNF        23007
JJ         22473
VF         12683
PRP        10474
VAUX        8128
RB          6852
DMD         5755
PSP         4883
CCS_UT      4827
QTO         4752
CSD         4416
NST         3852
VNG         2663
QTF         1478
VINF        1005
INTF         758
CCS          466
NNV          438
SYM          242
DMQ           51
PRQ           18
VN            11
PRF            5
NEG            5
RPD            1
dtype: int64


In [ ]:
train_data = []

for index, row in data1.iterrows():
    text = row['name']

    # Check if 'tags' column is NaN, and skip the row if it is
    if pd.isna(row['tags']):
        continue

    entities = {'entities': []}
    tags = row['tags']

    start = None
    for i, tag in enumerate(tags):
        if tag.startswith('B-'):
            if start is not None:
                entities['entities'].append((start, i, tags[i-1][2:]))
            start = i
        elif tag.startswith('I-'):
            continue
        else:  # O tag or other
            if start is not None:
                entities['entities'].append((start, i, tags[i-1][2:]))
            start = None

    train_data.append((text, entities))


In [ ]:
!python -m spacy download xx_ent_wiki_sm


2023-12-15 14:15:18.128606: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-15 14:15:18.128673: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-15 14:15:18.130931: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-15 14:15:19.380378: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.1/11.1 MB 28.6 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('xx_ent_wiki_sm')


In [ ]:
import spacy.cli
spacy.cli.download("en_core_web_sm")


✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [ ]:
!python -m spacy download en_core_web_sm


2023-12-15 14:16:05.939753: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-15 14:16:05.939835: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-15 14:16:05.943168: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-15 14:16:07.516531: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 20.0 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [ ]:
import spacy

# Load the small multilingual spaCy model
nlp = spacy.load("xx_ent_wiki_sm")

In [ ]:
# Specify the name of the new directory
new_directory_name = 'output'

# Create the new directory
!mkdir $new_directory_name


In [ ]:
spacy.prefer_gpu()


False

In [ ]:
# Import necessary libraries
import spacy

# Load the spaCy model (replace 'en_core_web_sm' with the appropriate model)
nlp = spacy.load('en_core_web_sm')

# Define the number of training iterations
num_iterations = 100  # Adjust this number as needed

# Get the ner component of the pipeline
ner = nlp.get_pipe("ner")

# Prepare and format training data (replace train_data with your actual training data)
train_data = [
    ("Your text here", {"entities": [(0, 4, "LABEL")]}),
    # Add more examples as needed
]

# Train the model
# Train the model
for iteration in range(num_iterations):
    # Iterate over the training data
    for text, annotations in train_data:
        # Update the Named Entity Recognition model
        example = spacy.training.example.Example.from_dict(nlp.make_doc(text), annotations)
        nlp.update([example], drop=0.5)

    # Print some information after each iteration
    print(f"Iteration {iteration + 1} completed.")

# Save the trained model to a file
nlp.to_disk("output_model")
print("Training completed. Model saved.")



Iteration 1 completed.
Iteration 2 completed.
Iteration 3 completed.
Iteration 4 completed.
Iteration 5 completed.
Iteration 6 completed.
Iteration 7 completed.
Iteration 8 completed.
Iteration 9 completed.
Iteration 10 completed.
Iteration 11 completed.
Iteration 12 completed.
Iteration 13 completed.
Iteration 14 completed.
Iteration 15 completed.
Iteration 16 completed.
Iteration 17 completed.
Iteration 18 completed.
Iteration 19 completed.
Iteration 20 completed.
Iteration 21 completed.
Iteration 22 completed.
Iteration 23 completed.
Iteration 24 completed.
Iteration 25 completed.
Iteration 26 completed.
Iteration 27 completed.
Iteration 28 completed.
Iteration 29 completed.
Iteration 30 completed.
Iteration 31 completed.
Iteration 32 completed.
Iteration 33 completed.
Iteration 34 completed.
Iteration 35 completed.
Iteration 36 completed.
Iteration 37 completed.
Iteration 38 completed.
Iteration 39 completed.
Iteration 40 completed.
Iteration 41 completed.
Iteration 42 completed.
I

In [ ]:
# Load the trained model from disk
nlp = spacy.load("output_model")

# Test the trained model on new text
test_text = "Your test text here"
doc = nlp(test_text)

# Print recognized entities and their labels
for ent in doc.ents:
    print(f"Entity: {ent.text}, Label: {ent.label_}")


Entity: Your, Label: LABEL


In [ ]:
text = "குடிமகன் பள்ளிக்கு செல்ல உழைந்தான்."
doc = nlp(text)
for ent in doc.ents:
    print(ent.text, ent.label_)

In [ ]:
!pip install googletrans==4.0.0-rc1


  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 7.3 MB/s eta 0:00:00
  Created wheel for googletrans: filename=googletrans-4.0.0rc1-py3-none-any.whl size=17396 sha256=99085a147adb42a6b8e05d7e731d6b1d55bdaca26ce2a249ed658be5cb2a5a5f
  Stored in directory: /root/.cache/pip/wheels/c0/59/9f/7372f0cf70160fe61b528532e1a7c8498c4becd6bcffb022de
Successfully built googletrans
  Attempting uninstall: chardet
    Found existing installation: chardet 5.2.0
    Uninstalling c

In [ ]:
from googletrans import Translator

def translate_tamil_to_english(tamil_text):
    # Create a Translator object
    translator = Translator()

    # Translate tamil text to English
    translation = translator.translate(tamil_text, src='ta', dest='en')

    # Return the translated text
    return translation.text

# Example tamil text
tamil_text = "குடிமகன் பள்ளிக்கு செல்ல உழைந்தான்."

# Translate tamil to English
english_translation = translate_tamil_to_english(tamil_text)

# Print the results
print("Tamil:", tamil_text)
print("English Translation:", english_translation)


Tamil: குடிமகன் பள்ளிக்கு செல்ல உழைந்தான்.
English Translation: The citizen worked to go to school.


In [ ]:
import spacy

def identify_grammatical_roles(english_text):
    nlp = spacy.load("en_core_web_sm")
    doc = nlp(english_text)

    grammatical_roles = [(token.text, token.dep_) for token in doc]
    return grammatical_roles

# Example Tamil text
tamil_text = "குடிமகன் பள்ளிக்கு செல்ல உழைந்தான்."

# Translate Tamil to English
english_translation = translate_tamil_to_english(tamil_text)

# Identify grammatical roles in English
english_grammatical_roles = identify_grammatical_roles(english_translation)

# Print the results
print("Tamil:", tamil_text)
print("English Translation:", english_translation)
print("Grammatical Roles in English:", english_grammatical_roles)


Tamil: குடிமகன் பள்ளிக்கு செல்ல உழைந்தான்.
English Translation: The citizen worked to go to school.
Grammatical Roles in English: [('The', 'det'), ('citizen', 'nsubj'), ('worked', 'ROOT'), ('to', 'aux'), ('go', 'advcl'), ('to', 'prep'), ('school', 'pobj'), ('.', 'punct')]
